### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

### 安装特定依赖

In [ ]:
!cd modelscope-agent && pip install -r requirements.txt

### 本地配置

In [ ]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
os.chdir('modelscope-agent/examples/memory & knowledge')
sys.path.append('../../')

### 导入llama_index相关依赖

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    Settings
)
from modelscope import snapshot_download

### 下载知识库文件

In [ ]:
!mkdir -p 'demo/data/xianjiaoda/'
!wget 'https://modelscope.oss-cn-beijing.aliyuncs.com/resource/rag/xianjiaoda.md' -O 'data/xianjiaoda/xianjiaoda.md'

### 导入知识库文件

In [ ]:
# load documents
documents = SimpleDirectoryReader("demo/data/xianjiaoda/").load_data()
documents

### 下载embedding文件并且配置

In [ ]:
embedding_name='damo/nlp_gte_sentence-embedding_chinese-base'
local_embedding = snapshot_download(embedding_name)
Settings.embed_model = "local:"+local_embedding

### 构建Index

In [ ]:
index = VectorStoreIndex.from_documents(documents)

### API_KEY管理

In [ ]:
import os

print('请输入DASHSCOPE_API_KEY')
os.environ['DASHSCOPE_API_KEY'] = input()

### 构建agent

In [ ]:
from modelscope_agent.agents import RolePlay

role_template = '知识库查询小助手，可以优先通过查询本地知识库来回答用户的问题'
llm_config = {
    'model': 'qwen-max', 
    'model_server': 'dashscope',
    }
function_list = []

bot = RolePlay(function_list=function_list,llm=llm_config, instruction=role_template)

### 执行agent

In [ ]:
index_ret = index.as_retriever(similarity_top_k=3)
query = "西安交通大学图书馆有几部分组成？"
result = index_ret.retrieve(query)
ref_doc = result[0].text
response = bot.run("西安交通大学图书馆有几部分组成？", remote=False, print_info=True, ref_doc=ref_doc)
text = ''
for chunk in response:
    text += chunk
print(text)